# Index-objecten
zie ook https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html
## Een index bevat een immutable NumPy array
Een Index is te vergelijken met een NumPy array. Maar de array is *immutable* (de elementen kunnen niet gewijzigd worden.)

In [1]:
import pandas as pd
index = pd.Index([2, 6, 9])
print(f'{index[0]=}')
print(f'{index.size=}, {index.shape=}, {index.ndim=}, {index.dtype=}')
try:
    index[0] = 1
except TypeError as err:
    print('error:', err)

index[0]=np.int64(2)
index.size=3, index.shape=(3,), index.ndim=1, index.dtype=dtype('int64')
error: Index does not support mutable operations


## Een index voor een Series
Zoals we al gezien hebben, kunnen we een eigen index definiëren voor een Series. Dat moet geen getal zijn.

In [2]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=['a', 'b', 'c', 'd'])
print(data)
print(data.index)
print(f"{data['b']=}")

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64
Index(['a', 'b', 'c', 'd'], dtype='object')
data['b']=np.float64(0.5)


## "Willekeurige" waarden voor een index
Een index kan eender welke gegevens bevatten. Het moeten zelfs geen opeenvolgende waarden zijn. 

In [3]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0], index=[2, 5, 3, 7])
print(data)
print(data.index)
print(f"{data[5]=}")

2    0.25
5    0.50
3    0.75
7    1.00
dtype: float64
Index([2, 5, 3, 7], dtype='int64')
data[5]=np.float64(0.5)


## Indexen en slicing
We kennen indexen ook van NumPy arrays (en Python lists). Hoe zit het met de *slicing*-mogelijkheden? Valt je hier iets op (in vergelijking met NumPy arrays en Python lists)?

In [4]:
import pandas as pd
steden = {"Antwerpen": 560_000, "Mechelen": 90_000, "Leuven": 100_000, "Gent": 260_000, "Brugge": 120_000, "Hasselt": 90_000}
data = pd.Series(steden)
print("data['Mechelen':'Gent']", data['Mechelen':'Gent'], sep='\n')

data['Mechelen':'Gent']
Mechelen     90000
Leuven      100000
Gent        260000
dtype: int64


## Indexen kunnen gecombineerd worden met 'set-operaties'
We kunnen de gemeenschappelijke elementen (intersection) van twee indexen opvragen. Maar ook de twee sets samenvoegen(union). Wanneer we kiezen voor 'difference' trekken we de ene set af van de andere. (we houden de elementen over die niet in de andere set zitten). Een laatste mogelijkheid is de 'symmetric_difference': we houden de elementen over die niet gemeenschappelijk zijn.

In [5]:
import pandas as pd
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])
print(f"{indA.intersection(indB)=}")
print(f"{indA.union(indB)=}")
print(f"{indA.difference(indB)=}")
print(f"{indA.symmetric_difference(indB)=}")

indA.intersection(indB)=Index([3, 5, 7], dtype='int64')
indA.union(indB)=Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')
indA.difference(indB)=Index([1, 9], dtype='int64')
indA.symmetric_difference(indB)=Index([1, 2, 9, 11], dtype='int64')


## Testen of een waarde voor komt in een index
Met de 'in'-operator kunnen we controleren of een waarde aanwezig is in een index

In [1]:
import pandas as pd
ind = pd.Index([1, 3, 5, 7, 9])
for getal in (2, 3):
    if getal in ind:
        print(f"{getal} komt voor in {ind}")
    else:
        print(f"{getal} komt niet voor in {ind}")

2 komt niet voor in Index([1, 3, 5, 7, 9], dtype='int64')
3 komt voor in Index([1, 3, 5, 7, 9], dtype='int64')


## Dataframes vergelijken
Ik begin met twee bestanden te downloaden (als ze nog niet gedownload zijn.)

In [ ]:
from pathlib import Path
import requests
import pandas as pd
BI_CLEANED_CSV = 'bi_cleaned.csv'
BI_CSV = 'bi.csv'
bi_cleaned_path = Path(BI_CLEANED_CSV)
if not bi_cleaned_path.exists():
    data = requests.get('https://learn.walsoftcomputers.com/csv/cleaned_bi.csv')
    with open(BI_CLEANED_CSV, mode='wb') as f:
        f.write(data.content)
else:
    print(f"{BI_CLEANED_CSV} bestaat al.")

bi_path = Path(BI_CSV)
if not bi_path.exists():
    data = requests.get('https://learn.walsoftcomputers.com/csv/bi.csv')
    with open(BI_CSV, mode='wb') as f:
        f.write(data.content)
else:
    print(f"{BI_CSV} bestaat al")

bi_cleaned.csv bestaat al.
bi.csv bestaat al


Omdat de achternamen niet uniek zijn, maken we een index die bestaat uit twee kolommen (een multi-index). Aan de drop()-functie geven we twee indexwaarden: een voornaam (fNAME) en een achternaam (lNAME).

De functie symmetric_difference bevat de elementen die in één van de twee indexen voorkomen, maar niet in beide. 

De functie index1.difference(index2) bevat de elementen van index1 die niet in index2 voorkomen. 

In [15]:
bi_cleaned = pd.read_csv(BI_CLEANED_CSV, encoding='windows-1252', index_col=('fNAME', 'lNAME') )
bi = pd.read_csv(BI_CSV, encoding='windows-1252', index_col=('fNAME', 'lNAME'))
print('Er is geen verschil tussen beide indexen (summetric difference is leeg)')
print(bi_cleaned.index.symmetric_difference(bi.index))
print('\nVerwijder rij van Philip Leo')
bi_cleaned = bi_cleaned.drop(index=('Philip', 'Leo'))
print('\nsymmetric difference na drop van Philip Leo in bi_cleaned')
print(bi_cleaned.index.symmetric_difference(bi.index), end='\n\n')
print("bi.index.difference(bi_cleaned.index)(1 rij):")
print(bi.index.difference(bi_cleaned.index), end='\n\n')
print("bi_cleaned.index.difference(bi.index) (leeg):")
print(bi_cleaned.index.difference(bi.index))

Er is geen verschil tussen beide indexen (summetric difference is leeg)
MultiIndex([], names=['fNAME', 'lNAME'])

Verwijder rij van Philip Leo

symmetric difference na drop van Philip Leo in bi_cleaned
MultiIndex([('Philip', 'Leo')],
           names=['fNAME', 'lNAME'])

bi.index.difference(bi_cleaned.index)(1 rij):
MultiIndex([('Philip', 'Leo')],
           names=['fNAME', 'lNAME'])

bi_cleaned.index.difference(bi.index) (leeg):
MultiIndex([], names=['fNAME', 'lNAME'])
